In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4

In [2]:
from pyspark import SparkContext, SparkFiles

import os
from pathlib import Path

import requests
from bs4 import BeautifulSoup

import PyPDF2
import re
from collections import Counter

In [3]:
def get_books(url, target):
    if not os.path.isdir(target):
        os.makedirs(target)

    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    for a in soup.find_all("a", href=True):
        arr = a["href"].split(".")
        if arr[-1] == "pdf":
            response = requests.get("http:" + a["href"])
            with open(Path(target, a.string + ".pdf"), "wb+") as f:
                f.write(response.content)

In [4]:
def save_text_from_pdf(name):
    fileReader = PyPDF2.PdfFileReader(name)
    with open(Path(name + ".txt"), "a+") as f:
        for page in fileReader.pages:
            f.write(page.extractText())

In [5]:
def words_frequency(page_text):
    words = {}
    words_list = re.findall(r"\b[^\d\W]+\b", page_text)
    print(page_text)
    for word in words_list:
        if word in words:
            words[word] += 1
        else:
            words[word] = 1
    return words

In [6]:
def most_popular_words(name):
    words = words_frequency(name)
    print(words)
    k = Counter(words)
    return k.most_common(10)

In [7]:
folderName = 'freud'
get_books("https://holybooks.com/sigmund-freud-the-complete-works/", folderName)

p = Path(folderName)
print(p)
pdfFiles = list(p.glob('*.pdf'))
for b_p in pdfFiles:
    save_text_from_pdf(os.path.join(folderName, b_p.name))

sc = SparkContext("local", "first app")
p = Path(folderName)
allTextFiles = list(p.glob('*.txt'))
files = []
for b_p in allTextFiles:
    files.append(sc.textFile(os.path.join(folderName, b_p.name)).cache())

for file in files:
#     numXs = file.filter(lambda s: 'x' in s).count()
#     numYs = file.filter(lambda s: 'y' in s).count()
#     print(numXs, numYs)
    mostPopular = most_popular_words(file)
    print(mostPopular)


freud


TypeError: expected string or bytes-like object

In [8]:
from operator import add

In [10]:
for file in files:
#     numXs = file.filter(lambda s: 'x' in s).count()
#     numYs = file.filter(lambda s: 'y' in s).count()
#     print(numXs, numYs)
#     print(file.collect()[:10])
    a = file.map(lambda x : re.findall(r"\b[^\d\W]+\b", x))
    a = a.flatMap(lambda x: x)
    a = a.map(lambda x: (x,1))
    b = a.collect()
    k = Counter(b)
    print(k.most_common(10))
    print(b[:10])
    c =a.reduceByKey(add).collect()
    print(c)
    print(a.count())
    
#     mostPopular = most_popular_words(file)
#     print(mostPopular)

[(('the', 1), 131812), (('of', 1), 94372), (('to', 1), 60384), (('a', 1), 46599), (('in', 1), 46310), (('and', 1), 44579), (('that', 1), 33386), (('is', 1), 30321), (('which', 1), 22634), (('it', 1), 22311)]
[('Freud', 1), ('Complete', 1), ('Works', 1), ('Ivan', 1), ('Smith', 1), ('All', 1), ('rights', 1), ('reserved', 1), ('STUDIES', 1), ('ON', 1)]
[('Freud', 186), ('Complete', 2), ('Works', 6), ('Ivan', 4), ('Smith', 35), ('All', 341), ('rights', 50), ('reserved', 42), ('STUDIES', 3), ('ON', 92), ('HYSTERIA', 10), ('Intentionally', 150), ('left', 919), ('blank', 161), ('PREFACE', 28), ('TO', 104), ('THE', 472), ('FIRST', 5), ('EDITION', 6), ('In', 4553), ('we', 11462), ('published', 180), ('a', 46599), ('Preliminary', 27), ('Communication', 24), ('¹', 2750), ('on', 10518), ('new', 1325), ('method', 550), ('of', 94372), ('examining', 53), ('and', 44579), ('treating', 109), ('hysterical', 721), ('phenomena', 570), ('To', 500), ('this', 14730), ('added', 481), ('as', 18465), ('concisely